# 11. CODE SEARCH

1. Introduction
2. Bi-Encoder
3. Cross-Encoder
4. TOSS
5. References

# 1. Introduction

#### Зачем нужен поиск по коду?

- по текстовому описанию найти код, которые реализует указанный функционал
- по коду найти аналогичный или похожий код

#### Чем отличается поиск по коду от поиска по тексту?

- языки программирования отличаются от естественных языков
- код имеет вполне определённую структуру
- наличие разных языков в одном репозитории
- часто поисковый запрос и ответ на него написаны на разных языках

#### Поисковые запросы

Запрос (query) --- явное выражение поискового намерения, используемое пользователем.

Что хотим от языка запросов?
- Простота. Легко формулировать запрос.
- Выразительность. Язык запросов должен позволять определить то, что надо найти.
- Точность. Запросы должны позволять выразить намерение настолько однозначно, насколько это возможно.

Типы языков запросов:
1. Неформальные: в свободной форме
2. Формальные:
     - использующие общеизвестные языки программирования
     - использующие специально разработанные языки программирования
     - использующие пары: вход -- выход
3. Гибридные

#### Языки запросов в свободной форме

> Например, `read file line by line`

Запрос описывает намерение на естественном языке. Могут быть элементы языка программирования.

> Например, `FileReader close`

Преимущества:
- удобно пользователю

Недостатки
- естественные языки неоднозначные (например, *float*)
- словарь, используемый в запросах, может не соответствовать словарю кодовой базы (например, *array* и [.] в Python-е означают массив)

Проблему несоответствия словарей (например, поиск регулярных варажений) можно облегчить, если вложить слова естественного языка и слова языка программирования в общее векторное пространство (word2vec, CodeBERT). Либо делать предобработку и расширение запросов.

#### Запросы на основе общеизвестных языков программирования

Основные подходы:
- фрагменты кода. Например, запрос --- это код с частичной реализацией, которую надо дописать:
> ```try { File file = File . createTempFile (" foo " , " bar ") ; } catch ( IOException e ) { }```
- код с плейсхолдерами --- явно указаны пропущенные фрагменты кода:
>```public void actionClose ( JButton a , JFrame f ) { __CODE_SEARCH__ ; } ```
- код с некоторыми паттернами --- абстрактный шаблоны кода для поиска:
>```if (# = #) @ ;```

Недостатки:
- проблема парсить код существующими библиотеками (надо сначала сделать код синтаксически корректным)

Преимущества:
- можно использовать в рекомендательной системы во время написания кода (экономится время за запрос)

#### Запросы на основе специально разработанных языков программирования

**Использование логических языков программирования**

Предикаты, описывающие свойства кода. Например, найти пакет с классом, который называется `HelloWorld`:
>```package (? P , class , ? C ) , class (? C , name , HelloWorld )```

Иногда есть возможность задать более высокоуровневые свойства. Например, 
>```import count > 5 AND extends class FooBar```

**Существенные расширения существующих языков**

Например, поиск вложенных `if-else`:
>```$ ( if $$ else $ ) $ + ```

**Другие специальные языки**

Например, описание программы так, что описываются вычисления через граф вычислений. Можно искать независимо от языка.

#### Использование пары: вход -- выход

Подход интересен тем, что использует ключевую особенность кода --- исполнимость. Запрос через примеры описывает желаемое поведение. Например,
> для входа `abc@def.org` хотим на выходе иметь `abc`.

#### Гибридные запросы

Подходы комбинируют несколько вариантов, описанных выше. Например,
> ```sort playerScores in ascending order```, где `playerScores` --- переменная из кода.

#### Общая схема  поиска по коду

![](res/11_overview.png)

([Grazia Pradel - Code search A survey of techniques for finding code 2022](https://arxiv.org/abs/2204.02765))

- indexing: подготовка данных для последующего поиска (что индексируем, а что нет? какие связи остаются?)
- preprocessing & expansion: изменение запроса пользователя для улучшения качества поиска (использование истории поиска; замена слов в запросе на основе синонимов или эмбеддингов и т.д.)
- retrieval: извлечение кандидатов на основе запроса
- ranking: ранжирование (на основании расстояний, методами машинного обучения)
- pruning: фильтрация (по порогу, по количеству, убрать похожие)

Бенчмарки:

- CodeSearchNet
- CodeXGLUE
- Search4Net
- CoSQA

#### Подходы

Существующие подходы поиска по коду условно можно разделить на две группы:
1. методы на основе Information retrieval: они часто работают быстро, но иногда не совсем точны
2. методы на основе DL: могут показывать высокое качество, но обычно более медленные из-за использования больших моделей.

Среди DL-подходов, в свою очередь, можно выделить *биэнкодерные (bi-encoder)* и *кроссэнкодерные (cross-encoder)* решения.

# 2. Биэнкодерный подход

В биэнкодерном подходе входные данные (текстовый запрос, код) независимо вкладываются в векторное пространство.

![](./res/11_bi-encoder.png)

Таким образом, эмбеддинги для кода могут быть вычислены заранее, что ускоряет работу поиска.

#### Обозначения

Пусть
- $q$ --- поисковый запрос
- $C$ --- корпус кода (где происходит поиск)
- $M(\cdot, \cdot)$ --- функция соответствия (match) сниппета кода $c$ поисковому запросу $q$, чем больше значение фунции $M$, тем лучше $c$ соответствует $q$.

Тогда поиск по коду --- это поиск такого $c$ для запроса $q$, что значение $M(c, q)$ максимально:
$$max_{c \in C}M(c, q).$$

#### Подход

Пусть $c \in C$ --- сниппет кода из корпуса $C$.
Пусть $\Gamma'$ и $\Gamma''$ --- энкодеры для запросов и сниппетов кода, соответственно.
Тогда через $e_q$ и $e_c$ обозначим эмбеддинги для $q$ и $c$:
$$e_q = \Gamma'(q)$$
$$e_c = \Gamma''(c).$$
Кроме того, нам необходима некоторая функция похожести или функция расстояния $s_{bi}$ для векторов.
Например, косинус угла между векторами или евклидово расстояние:
$$s_{bi}: (e_q, e_c) \mapsto R.$$

Таким образом, имеем $$M_{bi}(\cdot, \cdot) = s_{bi}(\Gamma'(\cdot), \Gamma''(\cdot)).$$

Энкодеры $\Gamma'$ и $\Gamma''$ --- нейронные сети, предварительно обученные на парах (текст, код).

# 3. Кроссэнкодерный подход

В отличие от бинэнкодеров в случае кроссэнкодера механизм эттеншна выполняется на паре запрос-код, таким образом  происходит взаимодействие информации на естественном языке с информацией на языке программирования в процессе векторизации.

![](./res/11_cross-encoder.png)

За счёт этого кроссэнкодерные подходы могут достичь более высокую точность, чем биэнкодерные решения.
Но кроссэнкодерные подходы не могут создавать отдельные эмбеддинги для коды, необходимые для многих приложений.

#### Подход

В этом подходе поисковый запрос и сниппет кода объединяются (конкатенируются через специальный токен $<SEP>$) и являются входом для модели $\Gamma$, которая вычисляет кроссмодальную похожесть (энкодер+регрессия):

$$M_{cr}(\cdot, \cdot) = \Gamma([q, <SEP>, c]).$$

# 4. TOSS

Как объединить преимущества всех походов?

![](./res/11_toss_paper.png)

В работе [Revisiting Code Search in a Two-Stage Paradigm](https://arxiv.org/abs/2208.11274) авторы предложили
TOSS (TwO-Stage fuSion code Search framework) --- двухэтапный фреймворк для поиска по коду, который сочетает в себе преимущества различных методов поиска кода.
TOSS сначала использует модели на основе Information Retrieval и биэнкодерные модели для эффективного поиска небольшого количества лучших кандидатов, а затем использует кроссэнкодеры для более точного ранжирования.

#### Двухэтапный поиск по коду

1. Сначала находим набор сниппетов-кандидатов на основе функции $M_{recall}$.
2. Затем ранжируем набор кандидатов на основе функции $M_{rank}$.

![](./res/11_toss_overview.png)

Если одна модель на первом этапе.

1. $C_{sub} = \arg \max_{C' \subset C, |C'| = K} \sum_{c \in C'} M_{recall}(c, q)$ --- набор из $K$ кандидатов.

Функция $M_{recall}$ должна быть быстрой, но, возможно, неточной.

2. $c_{*}   = \arg \max_{c \in C_{sub}}                          M_{rank  }(c, q)$ --- итоговый сниппет.

Функция $M_{rank}$ может быть сложной.


Кроме того, первый этап ($M_{recall}$) может включать несколько $m$ различных способов поиска по коду. Полученные кандидаты объединяются и отправляются на второй этап:

$$C^{i}_{sub} = \arg \max_{C' \subset C, |C'| = K} \sum_{c \in C'} M^i_{recall}(c, q), \,\,\, 1 \leq i \leq m$$
$$C_{all} = \cup_{1 \leq i \leq m}C^i_{sub}$$

# 5. References

- [Grazia Pradel - Code search A survey of techniques for finding code 2022](https://arxiv.org/abs/2204.02765)
- [Васильев - Поиск и навигация в IDE](https://www.youtube.com/watch?v=rhMGmDfmMNo&list=PLjufDdskD5S6jwrQZP-ikUtNWnRj5xNuN&index=18)
- Birillo et al - Reflekt A library for compile-time reflection in Kotlin 2022
- Chai et al - Cross-domain deep code search with few-shot meta learning 2022
- Cheng Kuang - CSRS Code search with relevance matching and semantic matching 2022
- Eberhart McMillan - Generating clarifying questions for query refinement in source code search 2022
- Grazia et al - DiffSearch A scalable and precise search engine for code changes 2022
- Gu et al - Accelerating code search with deep hashing and code classification 2022
- Gu et al - Multimodal representation for neural code search 2022
- Haldar et al - A multi-perspective architecture for semantic code search 2021
- [Hu et al - Revisiting code search in a two-stage paradigm 2022](https://arxiv.org/abs/2208.11274)
- Li et al - CodeRetriever Unimodal and bimodal contrastive learning for code search 2022
- Liu et al - CodeMatcher Searching code based on sequential semantics of important query words 2022
- Liu et al - GraphSearchNet Enhancing GNNs via capturing global dependency for semantic code search 2022
- Shi et al - Enhancing semantic code search with multimodal contrastive learning and soft data augmentation 2022
- Shuai et al - Improving code search with co-attentive representation learning 2020
- Sun et al - Code search based on context-aware code translation 2022
- Sun et al - On the importance of building high-quality training datasets for neural code search 2022
- Villmow et al - Addressing leakage in self-supervised contextualized code retrieval 2022
- Wang et al - Enriching query semantics for code search with reinforcement learning 2021
- Yan et al - Are the code snippets what we are searching for A benchmark and an empirical study on code search with natural-language queries 2020
- Zhang et al - Bag-of-words baselines for semantic code search 2021
- https://ieeexplore.ieee.org/abstract/document/8453172
- https://dl.acm.org/doi/abs/10.1145/2393596.2393606
- https://ieeexplore.ieee.org/abstract/document/6606630